In [1]:
from games_setup import *
import SBMLLint.common.constants as cn
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [9]:
simple = load_file_from_curated_data(40)
m = MESGraph(simple)
for reaction in simple.reactions:
  print(reaction)
  print(reaction.category)

Reaction1: Br + BrO3 -> HBrO2 + HOBr; Br * BrO3 * k1 * BZ
reaction_n_n
Reaction2: Br + HBrO2 -> 2.00 HOBr; Br * HBrO2 * k2 * BZ
reaction_n_n
Reaction3: BrO3 + HBrO2 -> Ce + 2.00 HBrO2; BrO3 * HBrO2 * k3 * BZ
reaction_n_n
Reaction4: 2.00 HBrO2 -> BrO3 + HOBr; pow(HBrO2, 2) * k4 * BZ
reaction_n_n
Reaction5: Ce -> Br; Ce * k5 * BZ
reaction_1_1


In [11]:
{r.molecule for r in simple.reactions[2].reactants}

{BrO3, HBrO2}

In [87]:
m.nodes

NodeView(({Fruex}, {Fru}, {Glcex}, {Glc}, {ATP}, {HexP}, {ADP}, {UDP}, {Suc6P}, {Suc}, {phos}, {glycolysis}, {Sucvac}))

In [88]:
m.edges

OutEdgeView([])

In [89]:
for reaction in simple.reactions:
  if reaction.category == cn.REACTION_n_n:
    print(reaction)

v3: ATP + Glc -> HexP + ADP; compartment * (Vmax3 * (Glc / Km3Glc) * (ATP / Km3ATP) / ((1 + ATP / Km3ATP) * (1 + Glc / Km3Glc + Fru / Km4Fru + 0.113 * HexP / Ki3G6P + 0.0575 * HexP / Ki4F6P)))
v4: Fru + ATP -> HexP + ADP; compartment * (Vmax4 * (Fru / Km4Fru) * (ATP / Km4ATP) / ((1 + ATP / Km4ATP) * (1 + Glc / Km3Glc + Fru / Km4Fru + 0.113 * HexP / Ki3G6P + 0.0575 * HexP / Ki4F6P)))
v5: Fru + ATP -> HexP + ADP; compartment * (Vmax5 / (1 + Fru / Ki5Fru) * (Fru / Km5Fru) * (ATP / Km5ATP) / (1 + Fru / Km5Fru + ATP / Km5ATP + Fru * ATP / (Km5Fru * Km5ATP) + ADP / Ki5ADP))
v8: HexP + Fru -> Suc + UDP; compartment * (-Vmax8f * (Suc * UDP - Fru * 0.8231 * HexP / Keq8) / (Suc * UDP * (1 + Fru / Ki8Fru) + Km8Suc * (UDP + Ki8UDP) + Km8UDP * Suc + Vmax8f / (Vmax8r * Keq8) * (Km8UDPGlc * Fru * (1 + UDP / Ki8UDP) + 0.8231 * HexP * (Km8Fru * (1 + Km8UDP * Suc / (Ki8UDP * Km8Suc)) + Fru * (1 + Suc / Ki8Suc)))))


In [81]:
m.analyze(simple.reactions)

No cycles! No Type II Error!


{P0, P1, P2}->{CC, Cn}
{T0, T1, T2}->{CC, Cn}
{Mp};{Mt}

In [82]:
print(m.nodes)
print(m.edges)

[{Mp}, {Mt}, {P0, P1, P2}, {T0, T1, T2}, {CC, Cn}]
[({P0, P1, P2}, {CC, Cn}), ({T0, T1, T2}, {CC, Cn})]


In [7]:
INITIAL_NODES = 14
INITIAL_EDGES = 0
FINAL_NODES = 6
FINAL_EDGES = 6
UNIUNI0 = 0
UNIUNI1 = 7
UNIUNI2 = 8
UNIMULTI = 2
MULTIUNI = 13
INEQUAL1 = 14
INEQUAL2 = 15
AA = "AA"
CN = "Cn"
DFG = "DFG"
E1 = "E1"
E2 = "E2"
FRU = "Fru"
GLY = "Gly"
MEL = "Mel"
MG = "MG"

In [8]:
simple = load_file_from_curated_data(50)
m = MESGraph(simple)

In [9]:
molecules = [som.molecules for som in m.nodes]
{simple.getMolecule('MKKK')} in molecules

False

In [10]:
m.identifier

'{DFG};{E1};{E2};{Gly};{Cn};{_3DG};{FA};{_1DG};{AA};{Man};{Glu};{Mel};{MG};{Fru}'

In [11]:
# test constructor:
print(len(m.nodes) == INITIAL_NODES)
print(len(m.edges) == INITIAL_EDGES)
dfg = simple.getMolecule(DFG)
# molecules is a list of one-molecule sets
molecules = [som.molecules for som in m.nodes]
print({simple.getMolecule(DFG)} in molecules)

True
True
True


In [12]:
# testInitializeSOMs
for node in m.nodes:
  print(type(node) == SOM)

True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [13]:
# testMakeId
identifier = ""
for key, som in enumerate(m.nodes):
  identifier = identifier + som.identifier
  if key < len(m.nodes)-1:
    identifier = identifier + ";"
print(identifier == m.identifier)

True


In [14]:
# testGetNode
aa = simple.getMolecule(AA)
aa_som = m.getNode(aa)
print(type(aa_som) == SOM)
print(aa_som.molecules == {aa})

True
True


In [15]:
# testUniUniReaction
uniuni_reaction = simple.reactions[UNIUNI0]
m.processUniUniReaction(uniuni_reaction)
dfg = m.getNode(simple.getMolecule(DFG))
e1 = m.getNode(simple.getMolecule(E1))
print(m.has_node(dfg) == True)
print(m.has_node(e1) == True)
print(dfg == e1)

True
True
True


In [17]:
# testUniMultiReaction
unimulti_reaction = simple.reactions[UNIMULTI]
print(unimulti_reaction)
m.processUniMultiReaction(unimulti_reaction)
prods = [m.getNode(product.molecule) for product in unimulti_reaction.products]
dfg = m.getNode(simple.getMolecule(DFG))
for prod in prods:
  print(m.has_edge(prod, dfg))

v3: DFG -> Gly + Cn; k3 * DFG
True
True


In [18]:
m.edges

OutEdgeView([({Gly}, {DFG, E1}), ({Cn}, {DFG, E1})])

In [19]:
# testMultiUniReaction
multiuni_reaction = simple.reactions[MULTIUNI]
print(multiuni_reaction)
m.processMultiUniReaction(multiuni_reaction)
reacts = [m.getNode(reactant.molecule) for reactant in multiuni_reaction.reactants]
mel = m.getNode(simple.getMolecule(MEL))
for react in reacts:
  print(m.has_edge(react, mel))

v14: Gly + Cn -> Mel; k14 * Cn * Gly
True
True


In [20]:
m.edges

OutEdgeView([({Gly}, {DFG, E1}), ({Gly}, {Mel}), ({Cn}, {DFG, E1}), ({Cn}, {Mel})])

In [21]:
# testAddArc
source = [simple.getMolecule(FRU), simple.getMolecule(GLY)]
destination = [simple.getMolecule(E2)]
dummy_reaction = simple.reactions[INEQUAL2]
m.addArc(source, destination, dummy_reaction)
arc1 = [m.getNode(source[0]), 
        m.getNode(destination[0])]
arc2 = [m.getNode(source[1]), 
        m.getNode(destination[0])]
print(m.has_edge(arc1[0], arc1[1]))
print(m.has_edge(arc2[0], arc2[1]))
reaction_label1 = m.get_edge_data(arc1[0], arc1[1])[cn.REACTION][0]
reaction_label2 = m.get_edge_data(arc2[0], arc1[1])[cn.REACTION][0]
print(reaction_label1 == dummy_reaction.label)
print(reaction_label1 == reaction_label2)
print(m.edges)

True
True
True
True
[({Gly}, {DFG, E1}), ({Gly}, {Mel}), ({Gly}, {E2}), ({Cn}, {DFG, E1}), ({Cn}, {Mel}), ({Fru}, {E2})]


In [22]:
# test checkTypeOneError
uniuni_reaction1 = simple.reactions[7]
uniuni_reaction2 = simple.reactions[8]
inequality_reaction1 = simple.reactions[14]
inequality_reaction2 = simple.reactions[15]
m.processUniUniReaction(uniuni_reaction1)
m.processUniUniReaction(uniuni_reaction2)
aa = simple.getMolecule(AA)
cn = simple.getMolecule(CN)
mg = simple.getMolecule(MG)
print(m.checkTypeOneError((aa, cn), inequality_reaction1) == True)
print(m.checkTypeOneError((mg, aa), inequality_reaction2) == False)

We have a Type I Error...
AA  and  Cn  have the same weight by
v9: _1DG -> AA; k9 * _1DG
v8: _1DG -> Cn; k8 * _1DG

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  AA < Cn
We cannot add the arc:  AA -> Cn

True
True


In [30]:
# test checkTypeTwoError
REACTION1 = "Reaction2"
REACTION2 = "Reaction4"
simple2 = load_file_from_curated_data(40)
m2 = MESGraph(simple2)
m2.processMultiUniReaction(simple2.getReaction(REACTION1))
m2.processUniMultiReaction(simple2.getReaction(REACTION2))
print(m2.checkTypeTwoError() == True)
print(m2.type_one_error == False)
print(m2.type_two_error == True)

We have a Type II Error...

{HBrO2} < {HOBr} < {HBrO2} 

{HBrO2} -> {HOBr}  by
Reaction2

{HOBr} -> {HBrO2}  by
Reaction4

True
True
True


In [107]:
reaction = simple.reactions[16]
print(reaction)
rct = [a.stoichiometry for a in reaction.reactants]
pdt = [b.stoichiometry for b in reaction.products]
print(rct)
print(pdt)

v17: complexC -> Fus3 + Ste11 + Ste7 + Ste5; compartment * complexC * k17
[1.0]
[1.0, 1.0, 1.0, 1.0]


In [109]:
sum([])

0

In [108]:
sum(pdt)

4.0

In [39]:
m_analyze.nodes

NodeView(({MKKK, MKKK_P}, {MKK, MKK_P, MKK_PP}, {MAPK, MAPK_P, MAPK_PP}))

In [42]:
simple2.reactions

[Reaction1: Br + BrO3 -> HBrO2 + HOBr; Br * BrO3 * k1 * BZ,
 Reaction2: Br + HBrO2 -> 2.00 HOBr; Br * HBrO2 * k2 * BZ,
 Reaction3: BrO3 + HBrO2 -> Ce + 2.00 HBrO2; BrO3 * HBrO2 * k3 * BZ,
 Reaction4: 2.00 HBrO2 -> BrO3 + HOBr; pow(HBrO2, 2) * k4 * BZ,
 Reaction5: Ce -> Br; Ce * k5 * BZ]

In [40]:
# test analyze
simple1 = load_file_from_curated_data(50)
m_analyze = MESGraph(simple1)
m_analyze.analyze(simple1.reactions)
print(m_analyze.identifier)
print(len(m_analyze.nodes) == 6)
print(len(m_analyze.edges) == 6)
print(m_analyze.type_one_error == True)
print(m_analyze.type_two_error == False)

simple2 = load_file_from_curated_data(40)
m_analyze2 = MESGraph(simple2)
m_analyze2.analyze(simple2.reactions)
print(m_analyze2.type_one_error == False)
print(m_analyze2.type_two_error == True)

We have a Type I Error...
AA  and  Cn  have the same weight by
v5: _3DG -> Cn; k5 * _3DG
v6: _3DG -> FA; k6 * _3DG
v8: _1DG -> Cn; k8 * _1DG
v13: Glu -> _3DG; k13 * Glu
v12: Man -> Glu; k12 * Man
v9: _1DG -> AA; k9 * _1DG

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  AA < Cn
We cannot add the arc:  AA -> Cn

We have a Type I Error...
FA  and  Cn  have the same weight by
v5: _3DG -> Cn; k5 * _3DG
v6: _3DG -> FA; k6 * _3DG
v8: _1DG -> Cn; k8 * _1DG
v13: Glu -> _3DG; k13 * Glu
v12: Man -> Glu; k12 * Man
v9: _1DG -> AA; k9 * _1DG

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  FA < Cn
We cannot add the arc:  FA -> Cn

No cycles! No Type II Error!
{Gly}->{DFG, E1, E2}
{Gly}->{Mel}
{MG}->{AA, Cn, FA, Glu, Man, _1DG, _3DG}
{Fru}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{Mel}

True
True
True
True
We have a Type II Error...

{HOBr} < {HBrO2} < {HOBr} 

{HOBr} -> {HBrO2}  by
Reaction4

{HBrO2} -> {H

In [ ]:
cn.RE

In [ ]:
# reduceReaction: 
def reduce(cls, reaction):
  Reduces reaction using existing cls.soms
  :param Reaction reaction:
  :return reaction/False:     
  # flag that will show whether the reaction was reduced
  reduced = False
  if reaction.category != cn.REACTION_n_n:
    return reduced
  # Reduces the reaction by examining all SOMs
  for som in cls.soms:
    reactants_in = deque([mole_stoich for mole_stoich in  
                           reaction.reactants if 
                           mole_stoich.molecule in som.molecules])
    reactants_out = [mole_stoich for mole_stoich in  
                      reaction.reactants if 
                      mole_stoich.molecule not in som.molecules]
    products_in = deque([mole_stoich for mole_stoich in  
                          reaction.products if 
                          mole_stoich.molecule in som.molecules])
    products_out = [mole_stoich for mole_stoich in  
                     reaction.products if 
                     mole_stoich.molecule not in som.molecules]
      #
  while reactants_in and products_in:
    reactant = reactants_in[0]
    product = products_in[0]
    if reactant.stoichiometry > product.stoichiometry:
      reactants_in[0] = MoleculeStoichiometry(reactant.molecule,
                                            reactant.stoichiometry - product.stoichiometry)
      products_in.popleft()
    elif reactant.stoichiometry < product.stoichiometry:
      products_in[0] = MoleculeStoichiometry(product.molecule,
                                            product.stoichiometry - reactant.stoichiometry)
      reactants_in.popleft()
    else:
      reactants_in.popleft()
      products_in.popleft()
    reactants = list(reactants_in) + reactants_out
    products = list(products_in) + products_out
    #  
  if (len(reaction.reactants) > len(reactants)) | \
    (len(reaction.products) > len(products)):
    reduced = True
    reaction.reactants = reactants
    reaction.products = products
        
  if reduced:
    reaction.identifier = reaction.makeIdentifier()
    reaction.category = reaction._getCategory() 
    return reaction
  else:
    return reduced

In [ ]:
# processMultiMultiReactions:


In [29]:
reactions = []
for reaction in [r for r in simple.reactions if r.category!=cn.REACTION_BOUNDARY]:
  if (reaction.reactants[0].molecule.name == cn.EMPTYSET) | \
      (reaction.products[0].molecule.name == cn.EMPTYSET):
   continue
  else:
    reactions.append(reaction)
# for reaction in reactions:
#   print(reaction)
z = m.analyze(reactions)

We have a Type I Error...
AA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v12: Man -> Glu; k12 * Man
v9: _1DG -> AA; k9 * _1DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v6: _3DG -> FA; k6 * _3DG

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  AA < Cn
We cannot add the arc:  AA -> Cn

We have a Type I Error...
FA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v12: Man -> Glu; k12 * Man
v9: _1DG -> AA; k9 * _1DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v6: _3DG -> FA; k6 * _3DG

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  FA < Cn
We cannot add the arc:  FA -> Cn

